In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df_sample = pd.read_csv('../input/sample/sample_submission_wgBqZCk.csv')
df_sample.head()

In [ ]:
df_test = pd.read_csv('../input/steam-janta-hack/train.csv')
df_test.head()

In [ ]:
df_game = pd.read_csv('../input/game-overview/game_overview.csv')
df_game.head()

In [ ]:
df_game.title.value_counts().sum()

In [ ]:
df_game.dtypes

In [ ]:
df_game.publisher[4]

In [ ]:
df_game[df_game.publisher=='Valve '].title

In [ ]:
df = pd.read_csv('../input/steam-janta-hack//train.csv')
df.head()

In [ ]:
df.user_suggestion.value_counts()

In [ ]:
df.user_review[0]

In [ ]:
df.shape

In [ ]:
df_game.shape

In [ ]:
len(df_test.title.unique())

In [ ]:
df_game.title.value_counts().sum()

In [ ]:
df_game.head()

In [ ]:
df.head()

In [ ]:
# df = df_test
df = df.merge(df_game, on=['title'], how='inner')
df.head()

In [ ]:
df.user_review[1]

In [ ]:
import re
REPLACE_NO_SPACE = re.compile("[.;:!\'?,\"()\[\]]")
p = REPLACE_NO_SPACE.sub("", df.user_review[1].lower())
p

In [ ]:
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
p = REPLACE_WITH_SPACE.sub(" ", p)
p

In [ ]:
import nltk 
from keras.preprocessing.text import text_to_word_sequence
p = text_to_word_sequence(df.user_review[1])
p

In [ ]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
filtered_sentence=[]
for token in p:
    if token not in stop_words:
        filtered_sentence.append(token)
print(p)
print(' '.join(filtered_sentence))

In [ ]:
from tqdm import tqdm_notebook as tqdm
for review in tqdm(df.user_review):
    review_tokens = text_to_word_sequence(review)
    filtered_sentence=[]
    for token in review_tokens:
        if token not in stop_words:
            filtered_sentence.append(token)
    filtered_sentence = ' '.join(filtered_sentence)
    df[df.user_review==review].user_review = filtered_sentence
df.head()

In [ ]:
df.user_review[0]

** Handling Missing values in year column and cleaning user review columns **

In [ ]:
! pip install langdetect
! pip install googletrans

In [ ]:
from ast import literal_eval
from langdetect import detect
from nltk.stem import PorterStemmer
import spacy
from googletrans import Translator
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
df = pd.concat([df, pd.get_dummies(df.title)], 1)
df.head()

In [ ]:
title_mode = df.groupby('title')['year'].agg(pd.Series.mode).to_dict()
years = []
for i in df[['title','year']].values:
    if pd.isnull(i[1]):
        years.append(title_mode[i[0]])
    else:
        years.append(i[1])
df.year = years
df = pd.concat([df, pd.get_dummies(df.year)], 1)

In [ ]:
df.head()

In [ ]:
# dropping extra columns
df.drop(['title', 'year', 'developer', 'publisher', 'tags', 'overview'], 1, inplace=True)

In [ ]:
translator = Translator()
reviews = []
for i,j in tqdm(enumerate(df['user_review'].values)):
    try:
        lang = detect(j)
        if not lang == 'en':
            t = translator.translate(j, dest='en')
            reviews.append(t.text)
        else:
            reviews.append(j)
    except:
        reviews.append(j)

In [ ]:
stemmer = PorterStemmer()
import re
def clean(X):
    corpus = []
    for review in tqdm(X):
        review = re.sub(r'[^a-z0-9♥]', ' ', review.lower())
        review = re.sub(r'(♥+)', ' love ', review)
        clean_review = []
        for word in review.split():
            if len(word)>1:
                word = stemmer.stem(word)
                if len(word)>10:
                    word = word[:10]
                clean_review.append(word)
        corpus.append(' '.join(clean_review))
    return corpus

In [ ]:
reviews = clean(reviews)
print(len(reviews))
df['user_review'] = reviews

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
df.iloc[0,2] ## checking for user_suggestion column

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(binary=True)
cv.fit(df.user_review.values)
X = cv.transform(df.user_review.values)
Y = df.user_suggestion

In [ ]:
print(X.shape)
print(Y.shape)
print(df.shape)

In [ ]:
a = np.array([[1, 2], [3, 4]])
b = np.array([[5, 6, 7],[8, 9, 10]])
np.hstack((a, b))

In [ ]:
print(a.shape)
print(b.shape)

In [ ]:
print(X.shape)
print(df.drop(columns=['user_review', 'user_suggestion'], axis=1).values.shape)

In [ ]:
print(X.toarray())

In [ ]:
# X = np.hstack([X, df.drop(columns=['user_review', 'user_suggestion'], axis=1).values])
X = np.hstack((df.drop(columns=['user_review', 'user_suggestion'], axis=1).values, X.toarray()))
print(X.shape)


## Final Attempt

In [ ]:
df.head()

In [ ]:
title_mode = df.groupby('title')['year'].agg(pd.Series.mode).to_dict()
years = []
for i in df[['title','year']].values:
    if pd.isnull(i[1]):
        years.append(title_mode[i[0]])
    else:
        years.append(i[1])
df.year = years
categorical_cols = ['title', 'year', 'developer', 'publisher']
df = pd.get_dummies(df, columns=categorical_cols)
to_drop = ['tags', 'overview']
df.drop(columns=to_drop, axis=1, inplace=True)
df.head()

In [ ]:
from tqdm.notebook import tqdm
translator = Translator()
reviews = []
for i,j in tqdm(enumerate(df['user_review'].values)):
    try:
        lang = detect(j)
        if not lang == 'en':
            t = translator.translate(j, dest='en')
            reviews.append(t.text)
        else:
            reviews.append(j)
    except:
        reviews.append(j)
reviews = clean(reviews)
print(len(reviews))
df['user_review'] = reviews
df.head()

In [ ]:
df.to_csv('final_attempt_train.csv', index=False)

In [ ]:
df = pd.read_csv('final_attempt_train.csv')
# df.head()

In [ ]:
print(*df.isnull().sum())

In [ ]:
df.isnull().sum()

In [ ]:
df.dropna(how='any', inplace=True)
df.isnull().sum()

In [ ]:
df.dtypes

In [ ]:
for rev in df.user_review:
    if(type('str')!=type(rev)):
        print(rev)
df = df[df.user_review!=0]
for rev in df.user_review:
    if(type('str')!=type(rev)):
        print(rev)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(binary=True)
cv.fit(df.user_review.values)
X = cv.transform(df.user_review.values)
Y = df.user_suggestion

In [ ]:
print(X.shape)
print(Y.shape)
print(df.shape)
X = np.hstack((df.drop(columns=['review_id', 'user_review', 'user_suggestion'], axis=1).values, X.toarray()))
print(X.shape)

In [ ]:
X

## BAd code

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train, test = train_test_split(df)
print(train.shape, test.shape)
x_train = train.drop(['user_suggestion'], 1).values
y_train = train['user_suggestion'].values
x_test = test.drop('user_suggestion', 1).values
y_test = test['user_suggestion'].values
print(x_train.shape, y_train.shape, x_test.shape)

In [ ]:
null_rows = np.where(x_test[:, 0]=='')[0]
null_rows

In [ ]:
x_train

In [ ]:
x_train[0,1]

In [ ]:
x_test

In [ ]:
tfidf_param = {
    'lowercase':True,
    'stop_words':'english',
    'ngram_range':(1,2),
    'min_df':10,
    'max_df':0.97,
}
tfidf = TfidfVectorizer(**tfidf_param)
vocab = tfidf.fit_transform(x_train[:,1]).toarray()
test_vocab = tfidf.transform(x_test[:,1]).toarray()
print(vocab.shape, test_vocab.shape)

In [ ]:
# adding tfidf with other features
x_train = np.hstack([vocab, x_train[:,1:]])
x_test = np.hstack([test_vocab, x_test[:, 1:]])
print(x_train.shape, x_test.shape)

In [ ]:
x_train

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(x_train, y_train, train_size=0.9)
X_test = x_test
Y_test = y_test

In [ ]:
print(X_train[0])

In [ ]:
count=0
for i in x_train[0]:
    if (type(i)!=type(0.0)):
        print(i)
        print('\n*****count', count)
    count+=1

## Creating a Baseline Model using only user reviews

Creating unique id for each game

In [ ]:
col_to_drop = ['review_id', 'title', 'year', 'developer', 'publisher', 'tags', 'overview']
df.drop(columns=col_to_drop, axis=1, inplace=True)
df.head()

In [ ]:
import re
def apply_regex(para):
    REPLACE_NO_SPACE = re.compile("[.;:!\'?,\"()\[\]]")
    para = REPLACE_NO_SPACE.sub("", para.lower())
    REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
    para = REPLACE_WITH_SPACE.sub(" ", para)
    return para

In [ ]:
df = df.user_review.apply(apply_regex)
df.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(binary=True)
cv.fit(df.values)
X = cv.transform(df.values)

In [ ]:
df_train = pd.read_csv('../input/steam-janta-hack/train.csv') 
df_train.head()

In [ ]:
Y = df_train.user_suggestion
Y

In [ ]:
type(X)

In [ ]:
X.shape

In [ ]:
len(cv.get_feature_names())

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y.as_matrix(), train_size=0.75)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, train_size=0.8)
print(X_train.shape)
print(X_test.shape)
print(X_val.shape)
print(Y_train.shape)
print(Y_test.shape)
print(Y_val.shape)

In [ ]:
Y_train=Y_train.reshape(-1,1)
Y_test=Y_test.reshape(-1,1)
Y_val=Y_val.reshape(-1,1)
print(Y_train.shape)
print(Y_test.shape)
print(Y_val.shape)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
model.add(Dense(300, input_dim=cv.get_feature_names(), activation='relu'))
model.add(Dense(150, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train, Y_train, batch_size=256, epochs=1, validation_data=(X_val, Y_val))

In [ ]:
print(model.metrics_names)
print(model.evaluate(X_test, Y_test))

In [ ]:
import pickle
file_name = 'final_dnn_model.sav'
pickle.dump(model, open(file_name, 'wb'))


In [ ]:
## loading the model 
file_name = 'first_dnn_model.sav'
model = pickle.load(open(file_name,'rb'))

In [ ]:
model

* ## Using Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr = LogisticRegression()

In [ ]:
lr.fit(X_train, Y_train)

## Using Decision Trees

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dclf = DecisionTreeClassifier() 
dclf.fit(X_train, Y_train)

## Using RF classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf_clf = RandomForestClassifier(n_estimators=100)
rf_clf.fit(X_train, Y_train)

## Using XGBoost

In [ ]:
import xgboost as xgb
xgb_clf = xgb.XGBClassifier()
xgb_clf.fit(X_train, Y_train)

In [ ]:
df.head()

## Testing the model accuracy

In [ ]:
from sklearn.metrics import accuracy_score
# Y_predict = lr.predict(X_test)
# Y_predict = rf_clf.predict(X_test)
# Y_predict = xgb_clf.predict(X_test)
Y_predict = dclf.predict(X_test)

accuracy_score(Y_test, Y_predict)

In [ ]:
Y_predict = xgb_clf.predict(X_val)

accuracy_score(Y_val, Y_predict)

## Predicting on the test dataset

In [ ]:
df_test = pd.read_csv('../input/steam-janta-hack/test.csv')
df_test.head()

In [ ]:
df_test.drop(columns=['review_id', 'title', 'year'], axis=1, inplace=True)
df_test.head()

In [ ]:
def remove_stop_words(df):
    for review in tqdm(df.user_review):
        review_tokens = text_to_word_sequence(review)
        filtered_sentence=[]
        for token in review_tokens:
            if token not in stop_words:
                filtered_sentence.append(token)
        filtered_sentence = ' '.join(filtered_sentence)
        df[df.user_review==review].user_review = filtered_sentence
    return df

In [ ]:
# df_test = remove_stop_words(df_test)
df_test = df_test.user_review.apply(apply_regex)
df_test.head()

In [ ]:
X_test = cv.transform(df.user_review.values)

## in case of using other columns run the following code

In [ ]:
from tqdm.notebook import tqdm
df_test = pd.read_csv('../input/steam-janta-hack/test.csv')
df_test = df_test.merge(df_game, on=['title'], how='inner')
title_mode = df_test.groupby('title')['year'].agg(pd.Series.mode).to_dict()
years = []
for i in df_test[['title','year']].values:
    if pd.isnull(i[1]):
        years.append(title_mode[i[0]])
    else:
        years.append(i[1])
df_test.year = years
categorical_cols = ['title', 'year', 'developer', 'publisher']
df_test = pd.get_dummies(df_test, columns=categorical_cols)
to_drop = ['tags', 'overview']
df_test.drop(columns=to_drop, axis=1, inplace=True)
translator = Translator()
reviews = []
for i,j in tqdm(enumerate(df_test['user_review'].values)):
    try:
        lang = detect(j)
        if not lang == 'en':
            t = translator.translate(j, dest='en')
            reviews.append(t.text)
        else:
            reviews.append(j)
    except:
        reviews.append(j)
reviews = clean(reviews)
df_test['user_review'] = reviews        
df.dropna(how='any', inplace=True)
df.isnull().sum()
print(len(df.columns))
df.head()


# X_test = np.hstack((df_test.drop(columns=['user_review'], axis=1).values, X_test.toarray()))

In [ ]:
for col in df.columns:
    if col not in df_test.columns:
        print(col)
        df_test[col] = 0

In [ ]:
len(df_test.dtypes) ## this includes Sugesstion also

In [ ]:
extra_col=['user_suggestion']
for col in df_test.columns:
    if col not in df.columns:
        extra_col.append(col)
df_test.drop(columns=extra_col, axis=1, inplace=True)
len(df_test.dtypes)
        

In [ ]:
len(df.dtypes)

In [ ]:
df_test.to_csv('final_attempt_test.csv', index=False)

In [ ]:
df_test = pd.read_csv('final_attempt_test.csv')
df_test.head()

In [ ]:
df_test.drop(columns=['review_id'], inplace=True, axis=1)

In [ ]:
df_test.isna().sum()*100/df_test.shape[0]

In [ ]:
df_test.fillna(0, inplace=True)

In [ ]:
X_test = cv.transform(df_test.user_review.values)
X_test = np.hstack((df_test.drop(columns=['user_review'], axis=1).values, X_test.toarray()))
print(X_test.shape)

In [ ]:
X_test.shape

In [ ]:
X_test

In [ ]:
Y_predict = model.predict(X_test)
# Y_predict = lr.predict(X_test)

In [ ]:
Y_predict

In [ ]:
Y_predict.mean()

In [ ]:
df_sample = pd.read_csv('../input/sample/sample_submission_wgBqZCk.csv')
df_sample.head()

In [ ]:


df_sample.user_suggestion = Y_predict
df_sample.head()

In [ ]:
df_sample.user_suggestion.describe()

In [ ]:
for suggestion in tqdm(df_sample.user_suggestion):
    final_suggestion=0
    if suggestion >= 1:
        final_suggestion = 1
    df_sample[df_sample.user_suggestion==suggestion].user_suggestion = final_suggestion

In [ ]:
def modify(num):
    if num >= 1:
        return 1
    else:
        return 0

In [ ]:
df_sample.user_suggestion = df_sample.user_suggestion.apply(modify)

In [ ]:
df_sample.head()

In [ ]:
df_sample.to_csv('prediction_model_with_other_cols.csv', index=False)